**Progetto Machine Learning**
---
**Giovanni D'Agostino, Marco Giorgi**

Fase Preliminare: 
- individuare la GPU Tesla-P100
- montare il drive contenente il codice e i dataset

In [ ]:
########## Tipo GPU allocata #############
!nvidia-smi

Mon Dec 14 14:01:11 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
import sys
drive.mount('Project_ML')
sys.path.append('/content/Project_ML/My Drive/ML/ProveFinali/Fridge') 

Drive already mounted at Project_ML; to attempt to forcibly remount, call drive.mount("Project_ML", force_remount=True).


# **Load Dataset**



Fase di Caricamento:
- import librerie
- load dei dataset main e fridge
- descrizione base dei dataset


In [ ]:
import preprocessing 
import pandas as pd
import model
import metrics
import generator
import stats_data
import tensorflow as tf
import numpy as np

In [ ]:
pd.set_option('float_format', '{:f}'.format)
df_main = pd.read_csv("/content/Project_ML/My Drive/ML/ProveFinali/main_train.csv")
df_main.describe(include='all')

,timestamp,power
count,6082508,6082508.000000
unique,6082508,nan
top,2019-02-01 08:30:23+01:00,nan
freq,1,nan
mean,NaN,370.915554
std,NaN,549.188099
min,NaN,73.481000
25%,NaN,167.012000
50%,NaN,236.648000
75%,NaN,356.898000


In [ ]:
df_fridge = pd.read_csv("/content/Project_ML/My Drive/ML/ProveFinali/fridge_train.csv")
df_fridge.describe(include='all')

,timestamp,power
count,6082508,6082508.000000
unique,6082508,nan
top,2019-02-01 08:30:23+01:00,nan
freq,1,nan
mean,NaN,37.237106
std,NaN,46.988700
min,NaN,2.200000
25%,NaN,2.400000
50%,NaN,6.750000
75%,NaN,69.100000


# **Train**

Fase di Train: 
- configurazione size window e batch
- preprocessing (load + padding + standardizzazione )
- configurazione generatore e addestramento
- salvataggio modello

In [ ]:
window_size= 3999 # NB!deve avere dimensione dispari
batch_size = 512 #512

In [ ]:
main_power = preprocessing.load_data(df_main)
fridge_power= preprocessing.load_data(df_fridge)
mmM= np.mean(main_power)
stdM= np.std(main_power)
mmF= np.mean(fridge_power)
stdF= np.std(fridge_power)
#### main-> media:  370.9155542294131, deviazione: 549.1880538357854
#### fridge-> media:  37.23710644848696, deviazione: 46.988695952384774

In [ ]:
 ground_truth = fridge_power + np.zeros(fridge_power.shape)


In [ ]:
main_power_padded = preprocessing.padding(main_power, window_size) 
fridge_power_padded = preprocessing.padding(fridge_power, window_size)
main_power_strd= preprocessing.standardizzazione(main_power_padded, mmM, stdM)
fridge_power_strd = preprocessing.standardizzazione(fridge_power_padded, mmF, stdF)
generator = generator.DataGenerator(main_power_strd, fridge_power_strd, window_len=window_size, batch_len=batch_size)
model_fridge_tot = model.fridge_model(window_size)

In [ ]:
model_fridge_tot.fit(x=generator, epochs=3, steps_per_epoch = generator.__len__())

In [ ]:
# Salvataggio del modello #
model_fridge_tot.save('/content/Project_ML/My Drive/ML/ProveFinali/Fridge/modelFridge')

#**Test**

Fase di Test:
- import librerie
- caricamento modello
- predict
- reverse standardizzazione
- calcolo F1
- plot grafici

In [ ]:
import generator
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (30,10) # configurazione plot

In [ ]:
model= tf.keras.models.load_model('/content/Project_ML/My Drive/ML/ProveFinali/Fridge/modelFridge', custom_objects=None)

In [ ]:
data_generator = generator.DataGenerator(main_power_strd, fridge_power_strd, window_len=window_size, batch_len=batch_size, shuffle=False)
pred_fridge = model.predict(x = data_generator, steps=data_generator.__len__(), verbose=1)
pred_fridge_reverse= preprocessing.reverse_strd(mmF, stdF, pred_fridge)

In [ ]:
f1_p= metrics.f1_score(ground_truth, pred_fridge_reverse)
print("F1:",f1_p)

##**Plot**

In [ ]:
start_plot = 0   
end_plot = 3000000 # max 6082508

In [ ]:
fridge_power_plot=np.array(fridge_power).reshape(-1,1)
plt.plot(fridge_power_plot[start_plot:end_plot])
plt.plot(pred_fridge_reverse[start_plot:end_plot])